In [1]:
TEXT2SOURCEDICT = {}

In [103]:
import re
import pandas as pd
import os
def depersonalize(text):
    text = str(text)
    url_regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    text= re.sub(url_regex, "url", text)
    text = re.sub("id[\d]*",'',text)
    text = re.sub("> ?",'',text)
    text = re.sub("@[\w]*",'',text)
#     text = re.sub("\+[\d]*", "", text)
    text = re.sub("[\d]+", "NUMBER", text)
    text = re.sub(">>", "", text)
#     text = re.sub("[\d]{3,100}", "", text)
    text = re.sub("[\t|\n|\r]", "", text)
    text = re.sub("[(OP)|url|nickname|phone_number]", "", text)
    return text

In [3]:
# tr test

In [12]:
def get_texts_from_folder(folder, txt, sep):
    txt_set = set()
    for f in os.listdir(folder):
        pth = os.path.join(folder, f)
        df_t = pd.read_csv(pth, sep=sep)
        texts_set = set(df_t[txt].tolist())
        txt_set = txt_set.union(texts_set)
    return txt_set

In [5]:
train_tes_texts_set = get_texts_from_folder('./gr0_train/',"INPUT:text", '\t')

In [6]:
fpaths = ['g1.tsv','g2.tsv','g3.tsv','g1_4.tsv','g2_4.tsv','g3_4.tsv','maybe_old/g1.tsv','maybe_old/g2.tsv']
for f in fpaths:
    pth = './directly_from_toloka/' + f
    df_c = pd.read_csv(pth, sep = '\t')
    golden_columns = [c for c in df_c.columns if 'GOLDEN' in c]
    df_c = df_c[~df_c[golden_columns[1]].isna()]
    train_tes_texts_set = train_tes_texts_set.union(set(df_c['INPUT:text'].tolist()))

In [7]:
def add_set_to_dict(text_set, category):
    for t in text_set:
        if t not in TEXT2SOURCEDICT:
            TEXT2SOURCEDICT[t] = category
            
        t_pr = depersonalize(t)
        if t_pr not in TEXT2SOURCEDICT:
            TEXT2SOURCEDICT[t_pr] = category

In [8]:
add_set_to_dict(train_tes_texts_set, "trtest")

In [9]:
len(TEXT2SOURCEDICT)

912

In [ ]:
# toloka

In [10]:
toloka_set = get_texts_from_folder('./gr1_toloka/','text',',')

In [13]:
add_set_to_dict(toloka_set, "toloka")

In [14]:
len(TEXT2SOURCEDICT)

12939

In [15]:
# "self"

In [16]:
df1_s = pd.read_csv('./gr2_self/export_ready_st1 - Sheet1.csv')
df2_s = pd.read_csv('./gr2_self/SELF_LABEL_export_ready_st2_multilabel_view.csv')
df2_s = df2_s[:1000]
df_self = pd.concat([df1_s,df2_s])

In [17]:
self_list = set(df_self['text'].tolist())
len(self_list)

2009

In [18]:
add_set_to_dict(self_list, "self")
len(TEXT2SOURCEDICT)

15356

In [19]:
# kw

In [20]:
df2_s = pd.read_csv('./gr2_self/SELF_LABEL_export_ready_st2_multilabel_view.csv')
df2_s = df2_s[1000:]
df3_s = pd.read_csv('./gr2_self/SELF_LABLE_export_ready_st3_multilabel_view.csv')
df4_s = pd.read_csv('./gr2_self/SELF_LABLE_export_ready_st4_multilabel_view.csv')
df_kw = pd.concat([df2_s,df3_s,df4_s])
kw_list = set(df_kw['text'].tolist())

In [21]:
add_set_to_dict(kw_list, "kw")
len(TEXT2SOURCEDICT)

51849

In [22]:
# "kwolga"

In [23]:
df_kwolga_list = []
folder = './gr3_kw_olga/'
for f in os.listdir(folder):
    filepath = os.path.join(folder, f)
    df_curr = pd.read_csv(filepath, header = None)
    df_kwolga_list.append(df_curr)
    
df_kwolga = pd.concat(df_kwolga_list)

In [24]:
kwolga_set = set(df_kwolga[0].tolist())

In [25]:
add_set_to_dict(kwolga_set, "kwolga")
len(TEXT2SOURCEDICT)

51887

In [26]:
# Считаем

In [27]:
from collections import Counter

In [28]:
df_sensitive_published = pd.read_csv("../sensitive_topics/sensitive_topics.csv")

In [29]:
len(df_sensitive_published)

33303

In [30]:
source_list = []
not_found = 0
for t in df_sensitive_published['text'].tolist():
    t_proc = depersonalize(t)
    if t in TEXT2SOURCEDICT:
        source_list.append(TEXT2SOURCEDICT[t])
    elif t_proc in TEXT2SOURCEDICT:
        source_list.append(TEXT2SOURCEDICT[t_proc])
    else:
        not_found += 1
        source_list.append('toloka')
print(" {} marked as toloka but actually not found".format(not_found))

 365 marked as toloka but actually not found


In [36]:
Counter(source_list).most_common(100)

[('kw', 21656), ('toloka', 9291), ('self', 1978), ('trtest', 378)]

In [37]:
train_tes_texts_set_proc = set([depersonalize(t) for t in train_tes_texts_set])

In [38]:
Counter(df_sensitive_published['text'].isin(train_tes_texts_set_proc))

Counter({False: 32925, True: 378})

In [39]:
df_sensitive_published['source'] = source_list

In [40]:
df_sensitive_published.to_csv("sensitive_topics_vs_source.csv", index = None)

# Восстанавливаем после переработки

In [56]:
len(df_sensitive_published)

33303

In [43]:
df_sensitive_published.head()

,text,offline_crime,online_crime,drugs,gambling,pornography,prostitution,slavery,suicide,terrorism,weapons,body_shaming,health_shaming,politics,racism,religion,sexual_minorities,sexism,social_injustice,source
0,РРРРРРЯЯЯЯЯЯЯ РУССКИЕ ВАНЬКИ НИКОМУ НЕ НУЖНЫ!!...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,toloka
1,"По моему, быдло тот кто осуждает чужие предпоч...",0.0,0.0,0.0,0.0,1.0,0.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,toloka
2,Бабы это вообще безхуйные сиськастые ссущие си...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,trtest
3,изнасиловалиа говорила что девственницаЯ так д...,0.0,0.0,0.0,0.0,1.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,trtest
4,"и что ? не сосут только уебищные тни, видосов ...",0.0,0.0,0.0,0.0,1.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,toloka


# Присоединяем переразмеченную толоку

In [ ]:
df_toloka_relabeled = pd.read_csv("TOLOKA_low_conf_for_manual_relabeling_ready.csv")

In [57]:
df_sensitive_published_witout_dropped_toloka = df_sensitive_published[~df_sensitive_published['text'].isin(df_toloka_relabeled['text'])]

In [58]:
len(df_sensitive_published_witout_dropped_toloka)

31520

In [45]:
df_toloka_relabeled.head()

,text,vote1,vote2,vote3,offline_crime,online_crime,drugs,gambling,pornography,prostitution,...,body_shaming,health_shaming,politics,racism,religion,sexual_minorities,sexism,social_injustice,source,meet_req
0,"жирным трудно в армии, все жирики ходили в оби...",body_shaming,NaN,NaN,0.76,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.00,0.0,0.0,0.0,0.0,toloka,False
1,Америкосы думаете дураки что не возятся с КАПо...,politics,NaN,NaN,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.00,0.0,0.0,0.0,0.0,toloka,False
2,ополченцевСепаратисткое говно попросту грабану...,politics,NaN,NaN,1.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.79,0.0,0.0,0.0,0.0,toloka,False
3,Бендеры похитили и под дулом автомата заставил...,politics,NaN,NaN,1.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.00,0.0,0.0,0.0,0.0,toloka,False
4,А можно массово не выходить на работу Устраива...,terrorism,offline_crime,racism,1.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.00,0.0,0.0,0.0,0.0,toloka,False


In [73]:
df_toloka_relabeled_really_lbl = df_toloka_relabeled[~df_toloka_relabeled['vote1'].isna()]
df_toloka_relabeled_no_really = df_toloka_relabeled[df_toloka_relabeled['vote1'].isna()]

### Обрабатываем реально переразмеченную часть

In [49]:
df_toloka_relabeled_really_lbl = df_toloka_relabeled_really_lbl[['text','vote1','vote2','vote3']]

In [55]:
topics = list(df_sensitive_published.columns[1:-1])
len(topics)

18

In [64]:
collected_data = []
for i, el in df_toloka_relabeled_really_lbl.iterrows():
    assert type(el['vote1']) == str
    current_data = [el['text']]
    topics_list = [0] * len(topics)
    for vote in ['vote1','vote2','vote3']:
        if type(el[vote]) == str:
            voted_topic = el[vote]
            if voted_topic == 'none':
                break
            index_in_list = topics.index(voted_topic)
            topics_list[index_in_list] = 1
    current_data = current_data + topics_list + ['toloka_real_manual_relabel']
    collected_data.append(current_data)
    
df_toloka_relabeled_really_lbl_multiview = pd.DataFrame(data = collected_data, columns = df_sensitive_published.columns)

In [89]:
df_toloka_relabeled_really_lbl_multiview.sample(5)

,text,offline_crime,online_crime,drugs,gambling,pornography,prostitution,slavery,suicide,terrorism,weapons,body_shaming,health_shaming,politics,racism,religion,sexual_minorities,sexism,social_injustice,source
366,В мире бушуют войны люди моут от коронавируса ...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,toloka_real_manual_relabel
393,"Трезвым - тем более.Вот они, последствия дрочк...",0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,toloka_real_manual_relabel
340,ИЛИ Я ПОЖАЛУЮСЬ В ПОЛИЦИЮ!!!!!!!!Майор Петренк...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,toloka_real_manual_relabel
419,Тоже самое можно сказать и о мужиках. Они дела...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,toloka_real_manual_relabel
137,Где паРаша там нищита разруха кровь и тоталита...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,toloka_real_manual_relabel


### ту толоку которую не переразмечаем - просто дропаем те вероятности где не 1

In [67]:
len(df_toloka_relabeled_no_really)

1262

In [74]:
df_toloka_relabeled_no_really.drop(columns = ['vote1','vote2','vote3', 'meet_req'], inplace = True)

C:\Users\N.Babakov\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pandas\core\frame.py:4157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [75]:
df_toloka_relabeled_no_really.head()

,text,offline_crime,online_crime,drugs,gambling,pornography,prostitution,slavery,suicide,terrorism,weapons,body_shaming,health_shaming,politics,racism,religion,sexual_minorities,sexism,social_injustice,source
41,ну хз там не так часто пули летят даже если ле...,0.00,0.41,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.00,0.00,toloka
42,"Ходячие бомбы, паралич огонь, масляные лужи и ...",0.00,0.33,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.00,0.00,toloka
43,какол -не человек! увидел какла-убей! не прост...,1.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.00,0.81,toloka
44,международная преступная группировка под назва...,0.17,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.00,0.00,toloka
45,Ближайшая война может быть в Азии Латинской Ам...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.22,0.00,toloka


In [76]:
for t in topics:
    df_toloka_relabeled_no_really[t] = df_toloka_relabeled_no_really[t].apply(lambda x: 0 if x < 0.9 else 1)

<ipython-input-76-429ca9e424b8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_toloka_relabeled_no_really[t] = df_toloka_relabeled_no_really[t].apply(lambda x: 0 if x < 0.9 else 1)


In [78]:
df_toloka_relabeled_no_really['source'] = ['toloka_false_manual_relabel'] * len(df_toloka_relabeled_no_really)

<ipython-input-78-3e93707fefb7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_toloka_relabeled_no_really['source'] = ['toloka_false_manual_relabel'] * len(df_toloka_relabeled_no_really)


In [79]:
df_toloka_relabeled_no_really.head()

,text,offline_crime,online_crime,drugs,gambling,pornography,prostitution,slavery,suicide,terrorism,weapons,body_shaming,health_shaming,politics,racism,religion,sexual_minorities,sexism,social_injustice,source
41,ну хз там не так часто пули летят даже если ле...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,toloka_false_manual_relabel
42,"Ходячие бомбы, паралич огонь, масляные лужи и ...",0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,toloka_false_manual_relabel
43,какол -не человек! увидел какла-убей! не прост...,1,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,toloka_false_manual_relabel
44,международная преступная группировка под назва...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,toloka_false_manual_relabel
45,Ближайшая война может быть в Азии Латинской Ам...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,toloka_false_manual_relabel


In [82]:
refactored_sensitive_df = pd.concat([df_sensitive_published_witout_dropped_toloka,
                                    df_toloka_relabeled_really_lbl_multiview,
                                    df_toloka_relabeled_no_really])
refactored_sensitive_df.drop_duplicates(subset = ['text'],inplace = True)

In [85]:
# Sanity check

In [84]:
len(df_sensitive_published)

33303

In [83]:
len(refactored_sensitive_df)

33308

In [86]:
original_set = set(df_sensitive_published['text'])
refubrished_set = set(refactored_sensitive_df['text'])

In [87]:
len(original_set & refubrished_set)

33303

# Добавляем семплы из специфических источников но без ключевых слов (все размечены реально руками)

In [90]:
df_no_kw = pd.read_csv("no_keyword_spec_forums_for_self_label_ready.csv")

In [93]:
df_no_kw.drop(columns = ['topic'], inplace = True)

In [94]:
df_no_kw.head()

,processed,vote1,vote2
0,Удлинители бывают различных видов: металличес...,none,NaN
1,1 степень - способность к общению со снижением...,none,NaN
2,Просто получается что любви то у тебя и нет и...,none,NaN
3,Ну если ты за жрал гавно и кормил комаров то ...,social_injustice,NaN
4,... или я уже ничего не понимаю.,none,NaN


In [96]:
collected_data = []
for i, el in df_no_kw.iterrows():
    assert type(el['vote1']) == str
    current_data = [el['processed']]
    topics_list = [0] * len(topics)
    for vote in ['vote1','vote2']:
        if type(el[vote]) == str:
            voted_topic = el[vote]
            if voted_topic == 'none':
                break
            index_in_list = topics.index(voted_topic)
            topics_list[index_in_list] = 1
    current_data = current_data + topics_list + ['no_kw_manual_relabel']
    collected_data.append(current_data)
    
df_no_kw_multiview = pd.DataFrame(data = collected_data, columns = df_sensitive_published.columns)

In [102]:
df_no_kw_multiview.sample(3)

,text,offline_crime,online_crime,drugs,gambling,pornography,prostitution,slavery,suicide,terrorism,weapons,body_shaming,health_shaming,politics,racism,religion,sexual_minorities,sexism,social_injustice,source
505,Я вообще не очень понимаю абстракции. Их сложн...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,no_kw_manual_relabel
526,логично. То есть нам приходится понять их сист...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,no_kw_manual_relabel
553,"Дима, где ты это нашел?! Мля! Я выдержала две ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,no_kw_manual_relabel


In [104]:
df_no_kw_multiview['text'] = df_no_kw_multiview['text'].apply(depersonalize)

In [106]:
df_no_kw_multiview.sample(3)

,text,offline_crime,online_crime,drugs,gambling,pornography,prostitution,slavery,suicide,terrorism,weapons,body_shaming,health_shaming,politics,racism,religion,sexual_minorities,sexism,social_injustice,source
52,"Расскажи, пожалуйста, поподробнее. Вас туда ...",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,no_kw_manual_relabel
218,Хорошо выглядит. Лишний раз подтвержден...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,no_kw_manual_relabel
490,"Нет, Яр, при всей значимости влияния социума, ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,no_kw_manual_relabel


In [107]:
len(refactored_sensitive_df)

33308

In [108]:
df_sensitive_new = pd.concat([refactored_sensitive_df, df_no_kw_multiview])

In [109]:
len(df_sensitive_new)

33908

In [111]:
Counter(df_sensitive_new['source']).most_common(10)

[('kw', 21656),
 ('toloka', 7508),
 ('self', 1978),
 ('toloka_false_manual_relabel', 1262),
 ('no_kw_manual_relabel', 600),
 ('toloka_real_manual_relabel', 526),
 ('trtest', 378)]

In [112]:
df_sensitive_new.to_csv("sensitive_topics_vers3.csv",index = None)